In [ ]:
import tensorflow as tf
import random
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
#tf.enable_eager_execution()
#tf.compat.v1.enable_eager_execution()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
import time
from google.colab import files

In [ ]:
lms = []
count = 0

basepath = Path('/content/data/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
    files_in_subdir = item.iterdir()
    for item1 in files_in_subdir:
        files_in_subdir1 = item1.iterdir()
        for data in files_in_subdir1:
            loc = str(data)
            img = np.load(data)
            #img = cv2.imread(loc)
            img = cv2.resize(img, dsize=(64, 1000))
            if(count == 0):
                plt.imshow(img)
            count = count + 1
            img = (img-127.5)/127.5
            lms.append(img)

In [ ]:
print(np.shape(lms))
print(np.shape(lms[6]))
lms_array = np.asarray(lms)

In [ ]:
count = 0
faces = []
basepath = Path('/content/faces_data/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
    count = 0
    files_in_subdir = item.iterdir()
    for data in files_in_subdir:
        if(count == 0):
            faces.append(cv2.imread(str(data)))
        count = count+1
print(np.shape(faces))

In [ ]:
faces_data = np.zeros(shape = (1786, 128, 128, 3))
count = -1
j = -1
basepath = Path('/content/data/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
    count = count + 1
    files_in_subdir = item.iterdir()
    for item1 in files_in_subdir:
        files_in_subdir1 = item1.iterdir()
        for data in files_in_subdir1:
            j = j + 1
            faces_data[j] = faces[count]
faces_data = (faces_data - 127.5)/127.5

In [ ]:
trainX, testX, trainY, testY = train_test_split(lms_array, faces_data, test_size=0.15, random_state=1) 

In [ ]:
print(np.shape(trainX))
#print(np.shape(valX))
print(np.shape(testX))
print(np.shape(trainY))
#print(np.shape(valY))
print(np.shape(testY))

In [ ]:
def generator():
#inshape = (64, 1000, 64)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(1000,64), strides = 2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu', strides = 2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', strides = 2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
    model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
    model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
    model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
    model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
    model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))

    model.add(tf.keras.layers.Reshape((1, 1, 64)))

    model.add(tf.keras.layers.Conv2DTranspose(256, (4, 4), strides=(1, 1)))
    model.add(tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding = 'same', activation='relu')) 
    model.add(tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding = 'same', activation='relu'))
    model.add(tf.keras.layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding = 'same', activation='relu'))
    model.add(tf.keras.layers.Conv2DTranspose(8, (3, 3), strides=(2, 2), padding = 'same', activation='relu'))
    model.add(tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding = 'same', activation='relu'))  
    return model

In [ ]:
generator = generator()
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 499, 64)           12352     
_________________________________________________________________
batch_normalization (BatchNo (None, 499, 64)           256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 249, 128)          24704     
_________________________________________________________________
batch_normalization_1 (Batch (None, 249, 128)          512       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 124, 64)           24640     
_________________________________________________________________
batch_normalization_2 (Batch (None, 124, 64)           256       
_________________________________________________________________
average_pooling1d (AveragePo (None, 62, 64)            0

In [ ]:
def discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (4, 4), strides=(1, 1), input_shape=[128, 128, 3]))
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.LeakyReLU())
    
    model.add(tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))

    return model

In [ ]:
discriminator = discriminator()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 125, 125, 64)      3136      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 125, 125, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 256)       295168    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 30, 30, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 230400)           

In [ ]:
d_loss = []
g_loss = []
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def D_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    d_loss.append(total_loss)
    return total_loss
  
def G_loss(fake_output):
    loss =  cross_entropy(tf.ones_like(fake_output), fake_output)
    g_loss.append(loss)
    return loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(0.0007)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0007)

In [ ]:
def train_step(images1, images2):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(images1, training=True)
        real_output = discriminator(images2, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = G_loss(fake_output)
        disc_loss = D_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def sv_img(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions*127.5)+127.5
    predictions = predictions.numpy()
    plt.imshow(predictions[random.randrange(0, 267)].astype('uint8'))
    plt.show()

In [ ]:
count = 0
def train_GAN(dataset1, dataset2, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch1 in dataset1:
            for image_batch2 in dataset2:
                train_step(image_batch1, image_batch2)
                break
        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        #if(epoch%10 == 0):
        sv_img(generator, epoch + 1, testX)

In [ ]:
trainX = tf.convert_to_tensor(trainX , dtype=tf.float32)
#trainY = tf.convert_to_tensor(trainY , dtype=tf.float32)
testX = tf.convert_to_tensor(testX , dtype=tf.float32)

In [ ]:
n = 64
trainX = [trainX[i:i + n] for i in range(0, trainX.shape[0], n)]
print(np.shape(trainX[0]))

(64, 1000, 64)


In [ ]:
n = 64
trainY = [trainY[i:i + n] for i in range(0, trainY.shape[0], n)]
print(np.shape(trainY[0]))

(64, 128, 128, 3)


In [ ]:
final_model = train_GAN(trainX, trainY, 100)

In [ ]:
#final_model.save('/content/GAN.h5')
#files.download('/content/GAN.h5')